In [2]:
import pandas as pd

offical_trainX = pd.read_csv("./data/dengue_features_train.csv")
offical_trainY = pd.read_csv("./data/dengue_labels_train.csv")

trimmed_trainX=offical_trainX.drop(['week_start_date'], axis=1)
trimmed_trainY=offical_trainY.drop(['year', 'weekofyear'], axis=1)

sj_official_trainX= trimmed_trainX[trimmed_trainX.city == "sj"]
iq_official_trainX= trimmed_trainX[trimmed_trainX.city == "iq"]

sj_len_X=len(sj_official_trainX)
iq_len_X=len(iq_official_trainX)

sj_trainX=sj_official_trainX[:749]
iq_trainX=iq_official_trainX[:416]

sj_testX=sj_official_trainX[749:]
iq_testX=iq_official_trainX[416:]

sj_official_trainY= trimmed_trainY[trimmed_trainY.city == "sj"]
iq_official_trainY= trimmed_trainY[trimmed_trainY.city == "iq"]

sj_len_Y=len(sj_official_trainY)
iq_len_Y=len(sj_official_trainY)

sj_trainY=sj_official_trainY[:749]
iq_trainY=iq_official_trainY[:416]

sj_testY=sj_official_trainY[749:]
iq_testY=iq_official_trainY[416:]

trainX = pd.concat((sj_trainX, iq_trainX), axis=0)
trainY = pd.concat((sj_trainY, iq_trainY), axis=0)
testX = pd.concat((sj_testX, iq_testX), axis=0)
testY = pd.concat((sj_testY, iq_testY), axis=0)

trainY=trainY.drop('city', axis=1)
testY=testY.drop('city', axis=1)


In [ ]:
trainX.info()

In [ ]:
trainX.describe().T

In [ ]:
trainX.isnull().sum()

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
import warnings

warnings.filterwarnings("ignore")

all_numerical_features = trainX.select_dtypes(include=["int64", "float64"]).columns
all_categorical_features = trainX.select_dtypes(include=[object]).columns
numerical_features = [value for value in all_numerical_features]
categorical_features = [value for value in all_categorical_features]

total_abs_error = 0
runtime_amount = 15

for i in range(runtime_amount):

    # Preprocessing for numerical data
    numerical_transformer = Pipeline(
        steps=[("imputer", KNNImputer(n_neighbors=5)), 
               ("scaler", StandardScaler())]
    )
    # Preprocessing for categorical data
    categorical_transformer = ce.target_encoder.TargetEncoder()

    # Bundle Preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numerical_transformer, numerical_features),
            ("cat", categorical_transformer, categorical_features),
        ],
        remainder="passthrough",
    )

    model = RandomForestRegressor(n_estimators=100)

    pipeline = Pipeline([("preprocessor", preprocessor), ("regressor", model)])
    pipeline.fit(trainX, trainY)
    preds_Y = pipeline.predict(testX)

    absolute_error = mean_absolute_error(testY, preds_Y)
    total_abs_error += absolute_error
    print(i + 1, model, "MAE:", mean_absolute_error(testY, preds_Y))

print("Average MAE:", total_abs_error / runtime_amount)


1 RandomForestRegressor() MAE: 13.479621993127147
2 RandomForestRegressor() MAE: 13.553298969072166
3 RandomForestRegressor() MAE: 13.362783505154638
4 RandomForestRegressor() MAE: 13.549209621993127
5 RandomForestRegressor() MAE: 13.528109965635739
6 RandomForestRegressor() MAE: 13.591305841924397
7 RandomForestRegressor() MAE: 13.474329896907216
8 RandomForestRegressor() MAE: 13.400171821305841
9 RandomForestRegressor() MAE: 13.619106529209622
10 RandomForestRegressor() MAE: 13.477731958762886
11 RandomForestRegressor() MAE: 13.381408934707903
12 RandomForestRegressor() MAE: 13.554879725085911
13 RandomForestRegressor() MAE: 13.710309278350515
14 RandomForestRegressor() MAE: 13.475773195876286
15 RandomForestRegressor() MAE: 13.476116838487973
Average MAE: 13.508943871706755
